In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
dados_treino = pd.read_csv('../data/processed/train_data.csv')
dados_teste = pd.read_csv('../data/processed/test_data.csv')

In [3]:
le_cat1 = LabelEncoder()
le_cat2 = LabelEncoder()
le_cat3 = LabelEncoder()
le_brand = LabelEncoder()

le_cat1 = le_cat1.fit(dados_treino.category_1)
le_cat2 = le_cat2.fit(dados_treino.category_2)
le_cat3 = le_cat3.fit(dados_treino.category_3)
le_brand = le_brand.fit(dados_treino.brand_name)

dados_treino['category_1'] = le_cat1.transform(dados_treino.category_1)
dados_treino['category_2'] = le_cat2.transform(dados_treino.category_2)
dados_treino['category_3'] = le_cat3.transform(dados_treino.category_3)
dados_treino['brand_name'] = le_brand.transform(dados_treino.brand_name)

In [6]:
dic_cat1 = dict(zip(le_cat1.classes_, le_cat1.transform(le_cat1.classes_)))
dic_cat2 = dict(zip(le_cat2.classes_, le_cat2.transform(le_cat2.classes_)))
dic_cat3 = dict(zip(le_cat3.classes_, le_cat3.transform(le_cat3.classes_)))
dic_brand = dict(zip(le_brand.classes_, le_brand.transform(le_brand.classes_)))


In [29]:
dic_cat1

{'Beauty': 0,
 'Electronics': 1,
 'Handmade': 2,
 'Home': 3,
 'Kids': 4,
 'Men': 5,
 'No category': 6,
 'Other': 7,
 'Sports & Outdoors': 8,
 'Vintage & Collectibles': 9,
 'Women': 10}

In [39]:
dados_teste = pd.read_csv('../data/processed/test_data.csv')
dados_teste['category_1'] = dados_teste['category_1'].map(dic_cat1).fillna(-1)
dados_teste['category_2'] = dados_teste['category_2'].map(dic_cat2).fillna(-1)
dados_teste['category_3'] = dados_teste['category_3'].map(dic_cat3).fillna(-1)
dados_teste['brand_name'] = dados_teste['brand_name'].map(dic_brand).fillna(-1)
dados_teste

,name,category_1,category_2,category_3,item_condition_id,brand_name,price,shipping,item_description,date,stock
0,women s motorcycle tshirt,10,104,751.0,3,1709.0,6.0,1,xl fits like a small,9-4-2018,8
1,alex and ani necklace,10,58,530.0,2,47.0,11.0,1,path of life alex and ani necklace in gold has...,25-1-2018,8
2,steve madden red lace up heels,10,88,629.0,2,3682.0,36.0,0,never been worn isn t true to size runs small,30-11-2018,9
3,men s black tuxedo,5,94,811.0,3,2795.0,54.0,0,worn a few times single pleated pants pants ar...,30-7-2018,7
4,red open back mermaid style dress xs,10,35,334.0,1,2795.0,316.0,0,this full length wine red mermaid style backle...,29-7-2018,34
...,...,...,...,...,...,...,...,...,...,...,...
444494,pink vs sleep shorts,10,5,687.0,3,2928.0,12.0,0,size small great condition barely ever worn to...,16-12-2018,7
444495,coach hand bag,10,112,392.0,3,926.0,86.0,0,this large black hobo coach is in a very good ...,15-11-2018,3
444496,waffle knit sweater forever,10,97,226.0,3,1384.0,14.0,0,great condition tags forever american apparel ...,2-3-2018,4
444497,dr dre beats solo,1,100,381.0,3,457.0,69.0,0,blue fits any head amazing sound no longer in use,19-3-2018,6


In [20]:
rfr = RandomForestRegressor(n_jobs=-1, random_state=101)
X_treino = dados_treino[['category_1', 'category_2', 'category_3', 'brand_name', 'item_condition_id', 'shipping']]
y_treino = dados_treino['price']

rfr.fit(X_treino, y_treino)

RandomForestRegressor(n_jobs=-1, random_state=101)

In [45]:
amostra = train_test_split(dados_teste,
                           test_size=50_000,
                           random_state=101,
                           stratify=dados_teste[['category_1', 'item_condition_id']])[1]

In [46]:
pred = rfr.predict(amostra[['category_1', 'category_2', 'category_3', 'brand_name', 'item_condition_id', 'shipping']])

In [47]:
print('R² =', r2_score(amostra.price, pred))
print('RMSE =', mean_absolute_error(amostra.price, pred)**0.5)
print('MAE =', mean_absolute_error(amostra.price, pred))

R² = 0.3596908575944866
RMSE = 3.613969892051945
MAE = 13.060778380657949
